In [ ]:
import sklearn

In [ ]:
import pandas as pd
df = pd.read_csv("pima-indians-diabetes.csv")

In [ ]:
df.head()

,Pregancies,Glucose,BloodPressure,Skin Thickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## 1. Normal Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_1 = df.iloc[:,:8]
y_1 = df.iloc[:,8]

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size = 0.2, stratify = y_1)

Decision_Tree = DecisionTreeClassifier(max_depth = None,
                                       random_state = 1)

Decision_Tree.fit(X_train_1, y_train_1)

y_pred_1 = Decision_Tree.predict(X_test_1)
y_pred_train_1 = Decision_Tree.predict(X_train_1)
train_accuracy = accuracy_score(y_train_1, y_pred_train_1)
test_accuracy = accuracy_score(y_test_1, y_pred_1)
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 1.0
Test Accuracy: 0.6493506493506493


## 2. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the random forest classifier
rf = RandomForestClassifier(n_estimators=100, max_depth=None)
rf.fit(X_train_1, y_train_1)

# Predict on the test set
y_prediction = rf.predict(X_test_1)
y_prediction_train = rf.predict(X_train_1)

# Check accuracy
train_accuracy = accuracy_score(y_train_1, y_prediction_train)
test_accuracy = accuracy_score(y_test_1, y_prediction)
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Train Accuracy: 1.0
Test Accuracy: 0.7337662337662337


## 3. Neural Network

In [ ]:
import torch
from torch import nn

In [ ]:
np_array = df.values
data_tensor = torch.tensor(np_array, dtype=torch.float32)

In [ ]:
data_tensor.shape

torch.Size([768, 9])

In [ ]:
X_tensor = data_tensor[:,0:8]
y_tensor = data_tensor[:, 8]
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_tensor, y_tensor, test_size = 0.2, stratify = y_tensor)

In [ ]:
class Linear_Regression_model(nn.Module):
  def __init__(self, hidden_units : int):
    super().__init__()
    self.linear_layer =  nn.Sequential(
        nn.Linear(in_features = 8, out_features = hidden_units),
        nn.ReLU(),
        nn.Linear(in_features = hidden_units, out_features = hidden_units),
        nn.ReLU(),
        nn.Linear(in_features = hidden_units, out_features = 1),
        nn.Sigmoid()
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x)

torch.manual_seed(42)
model_1 = Linear_Regression_model(hidden_units = 32)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params = model_1.parameters(),
                            lr = 0.01)

In [ ]:
epochs = 1000

for epoch in range(epochs):
  model_1.train()
  y_pred = model_1(X_train_2)
  optimizer.zero_grad()
  loss = loss_fn(y_pred, y_train_2.unsqueeze(1))
  loss.backward()
  train_accuracy = accuracy_score(y_train_2, torch.round(y_pred).detach().numpy())
  optimizer.step()
  model_1.eval()
  with torch.inference_mode():
    if epoch%100 == 0:
      y_pred_test = model_1(X_test_2)
      test_accuracy = accuracy_score(y_test_2, torch.round(y_pred_test))
      test_loss = loss_fn(y_pred_test, y_test_2.unsqueeze(1))
      print(f"Epoch = {epoch} | Train loss = {loss} | Test loss = {test_loss} | Train Accuracy = {train_accuracy} | Test Accuracy = {test_accuracy}")

Epoch = 0 | Train loss = 0.7052377462387085 | Test loss = 0.7026256918907166 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 100 | Train loss = 0.6936607956886292 | Test loss = 0.6935781836509705 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 200 | Train loss = 0.6934172511100769 | Test loss = 0.6933648586273193 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 300 | Train loss = 0.693333089351654 | Test loss = 0.6932920813560486 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 400 | Train loss = 0.6932901740074158 | Test loss = 0.693255603313446 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 500 | Train loss = 0.6932641267776489 | Test loss = 0.6932335495948792 | Train Accuracy = 0.6514657980456026 | Test Accuracy = 0.6493506493506493
Epoch = 600 | Train loss = 0.6932465434074402 | Test loss = 0.6932188868522644 |

## 4. Some Other Techniques:




In [ ]:
df.describe()

,Pregancies,Glucose,BloodPressure,Skin Thickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.Outcome.value_counts()

,count
Outcome,
0,500
1,268


In [ ]:
X = df.drop('Outcome', axis = 'columns')
y = df.Outcome

In [ ]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
X_scaled = (X - X.mean())/X.std()
X_scaled.shape

(768, 8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify = y, random_state = 10)

In [ ]:
X_train.shape

(576, 8)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier(), X_scaled, y, cv = 5)
scores

array([0.68181818, 0.69480519, 0.70779221, 0.78431373, 0.75163399])

In [ ]:
scores.mean()

0.724072659366777

In [ ]:
## Using Bagging Classifier now
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

bag_model = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 200,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0
)

bag_model.fit(X_train, y_train)

bag_model.oob_score_

0.7673611111111112

In [ ]:
y_pred = bag_model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.7916666666666666

In [ ]:
scores = cross_val_score(bag_model, X, y, cv = 5)

In [ ]:
scores.mean()

0.7604957134368899

## 5. Trying to Implement XGBoost Classifier:

In [ ]:
import xgboost

xgboost = xgboost.XGBClassifier(learning_rate = 0.01, n_estimators = 500)

xgboost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = xgboost.predict(X_test)
accuracy_test = accuracy_score(y_pred, y_test)
accuracy_test

0.78125

In [ ]:
scores = cross_val_score(xgboost, X, y, cv = 5)
scores

array([0.75974026, 0.73376623, 0.74675325, 0.80392157, 0.75163399])

In [ ]:
scores.mean()

0.7591630591630592

## 6. Trying To Implement a VotingClassifier:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify = y, random_state = 10)

NameError: name 'X_scaled' is not defined

In [ ]:
#Trying to implement Voting/Stacking

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
m1 = KNeighborsClassifier()
m2 = LogisticRegression()
m3 = RandomForestClassifier(n_estimators=1000)
m4 = GaussianNB()
m5 = DecisionTreeClassifier()

In [ ]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier([("knn", m1),("lr", m2), ("rf", m3),("gn", m4), ("dt", m5)])
model.fit(X_train, y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier()),
                             ('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier(n_estimators=1000)),
                             ('gn', GaussianNB()),
                             ('dt', DecisionTreeClassifier())])

In [ ]:
y_pred = model.predict(X_test)
y_pred_2 = model.predict(X_train)
test_accuracy = accuracy_score(y_pred, y_test)
train_accuracy = accuracy_score(y_pred_2, y_train)
print(f"Train Accuracy : {train_accuracy}, Test Accuracy : {test_accuracy}")

Train Accuracy : 0.8871527777777778, Test Accuracy : 0.8020833333333334


In [ ]:
#We seem to be getting the best accuracy from the voting classifier that was coded above

## 7. Trying to Implement Boosting:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier(n_estimators = 9, random_state = 0)
abc.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=9, random_state=0)

In [ ]:
y_pred = abc.predict(X_test)
y_pred_2 = abc.predict(X_train)
test_accuracy = accuracy_score(y_pred, y_test)
train_accuracy = accuracy_score(y_pred_2, y_train)
print(f"Train Accuracy : {train_accuracy}, Test Accuracy : {test_accuracy}")

Train Accuracy : 0.7795138888888888, Test Accuracy : 0.8020833333333334


In [ ]:
#Adaboost and VotingClassifier are giving us the highest accuracies here.

In [ ]:
pip install --upgrade xgboost

In [ ]:
import xgboost

xgboost_model = xgboost.XGBClassifier(max_depth=5, n_estimators=4)

xgboost_model.fit(
    X_train, y_train,
    verbose=True
)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=4, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = xgboost_model.predict(X_test)
y_pred_2 = xgboost_model.predict(X_train)
test_accuracy = accuracy_score(y_pred, y_test)
train_accuracy = accuracy_score(y_pred_2, y_train)
print(f"Train Accuracy : {train_accuracy}, Test Accuracy : {test_accuracy}")

Train Accuracy : 0.8645833333333334, Test Accuracy : 0.7864583333333334


In [ ]:
0.7864583333333334
# here uptill now..
# xgboost < Adaboost < VotingClassifier

0.7864583333333334